## Handling Missing Data

In [1]:
import numpy as np
import pandas as pd

#### `None`: Pythonic missing data

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

In [3]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
98.9 ms ± 5.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
2.83 ms ± 327 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



In [4]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

This reflects the fact that addition between an integer and `None` is undefined.

#### `NaN`: Missing numerical data

In [5]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

dtype('float64')

In [6]:
1 + np.nan

nan

In [7]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

In [8]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

Keep in mind that `NaN` is specifically a floating-point value; there is no equivalent NaN value for integers, strings, or other types.

### Detecting null values
Pandas data structures have two useful methods for detecting null data: isnull() and notnull(). Either one will return a Boolean mask over the data. 

In [9]:
data = pd.Series([1, np.nan, 'hello', None])

In [10]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [11]:
data[data.notnull()]

0        1
2    hello
dtype: object

### Dropping null values
In addition to the masking used before, there are the convenience methods, dropna() (which removes NA values) and fillna() (which fills in NA values).

In [12]:
data.dropna()

0        1
2    hello
dtype: object

For a `DataFrame`, there are more options.

In [13]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


By default, `dropna()` will drop all rows in which any null value is present:

In [14]:
df.dropna()

,0,1,2
1,2.0,3.0,5


Alternatively, you can drop NA values along a different axis; `axis=1` drops all columns containing a null value:

In [15]:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


The default is `how='any'`, such that any row or column (depending on the axis keyword) containing a null value will be dropped. You can also specify `how='all'`, which will only drop rows/columns that are all null values:

In [16]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [17]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


For finer-grained control, the `thresh` parameter lets you specify a minimum number of non-null values for the row/column to be kept:

In [18]:
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


### Filling null values

In [19]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [20]:
#fill NA entries with a single value, such as zero:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [21]:
# forward-fill
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [22]:
# backword-fill
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [23]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [24]:
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0
